In [112]:
from typing import List, Self
import numpy as np
import random
import copy
from shogi_enum import Direction, PieceName, Promotion, Turn, TemplateMove, SquareStatus, ConsoleColor
from shogi_const import NUMBER_OF_EDGE, NUMBER_OF_SQUARES, MAXIMUM_NUMBER_OF_DIRECTIONS_DUE_TO_POSITIONAL_MOVEMENT, NUMBER_OF_DIRECTIONS, NUMBER_OF_PIECES
from shogi_class import Condition, Move, Piece, Square, WayOfMove, Board

In [105]:
c = Condition()

In [106]:
displayCondition(c)

|9|8|7|6|5|4|3|2|1|
| | | | | | | | | |1
| | | | | | | | | |2
| | | | | | | | | |3
| | | | | | | | | |4
| | | | | | | | | |5
| | | | | | | | | |6
| | | | | | | | | |7
| | | | | | | | | |8
|l| | | | | | | | |9
NotOnBoard:



In [107]:
for i in c.board.squares:
    if i.piece.piece.name != PieceName.NON:
        if i.address.row > 3: print(f"row:{i.address.row},column:{i.address.column},name:{i.piece.piece.pieceToString()}")

row:9,column:1,name:l


In [96]:
for i in c.board.squares:
    if i.piece.piece.name != PieceName.NON:
        print(i.piece.toString())

name:l,index:01,turn:1,direction:u0:0:8,u1:0:0,ul0:1:0,ul1:0:0,l0:2:0,l1:0:0,dl0:3:0,dl1:0:0,d0:4:0,d1:0:0,dr0:5:0,dr1:0:0,r0:6:0,r1:0:0,ur0:7:0,ur1:0:0,


In [115]:
list = c.serchPointableHands()
for ele in list:
    print(f"row:{ele.address.row},column:{ele.address.column},name:{ele.piece.piece.pieceToString()}")

row:1,column:1,name:L
row:1,column:1,name:L
row:1,column:1,name:L
row:1,column:1,name:L
row:1,column:1,name:L
row:1,column:1,name:L
row:1,column:1,name:L
row:1,column:1,name:L


In [116]:
for i in range(30):
    displayCondition(c)
    if i != 0:
        c.turn = not c.turn
        c.turNumber = i + 1
    
    pointableHands: List[Move] = serchPointableHands(c)
    fc: int = len(pointableHands)
    print(fc)
    x: int = random.randint(0, fc - 1)
    c = executeMove(c, pointableHands[x])
    if isEnd(c):
        if whichWin(c):
            print("first won")
        else:
            print("second won")
print("draw")

|9|8|7|6|5|4|3|2|1|
| | | | | | | | | |1
| | | | | | | | | |2
| | | | | | | | | |3
| | | | | | | | | |4
| | | | | | | | | |5
| | | | | | | | | |6
| | | | | | | | | |7
| | | | | | | | | |8
|l| | | | | | | | |9
NotOnBoard:

8


TypeError: list indices must be integers or slices, not NoneType